# Super "Newton" method

Newton's method corresponds to approximately solving $f(x)=0$ by doing a first-order Taylor approximation $f(x+\delta)\approx f(x)+\delta f'(x)$ and solving for the root of this *line* to get our step $\delta$.

Suppose that instead we do a Taylor expansion to *second* order, finding the root of a *quadratic*:

$$
f(x+\delta) \approx f(x) + f'(x) \delta + \frac{f''(x)}{2} \delta^2 = 0
$$

This is a quadratic equation, so we can use the quadratic formula to obtain:
$$
\delta =  - \frac{2f}{f' \pm \sqrt{(f')^2 - 2ff''}}
$$
where we've applied a couple of tricks to get the "right" solution:

1. We want to make sure our equation is reasonable as $f'' \to 0$, in which case we should get an ordinary Newton step.   This is a problem for the "ordinary" quadratic formula because the coefficient of $\delta^2$ is $f''$, and this goes in the denominator.  We can get a "better" quadratic formula for this case by dividing by $1/\delta^2$ and writing the quadratic equation for $1/\delta$.

2. The quadratic formula gives two roots.  We only want the *smaller* $\delta$ root, because we want to take small steps, and that determines the sign.  This means that we should choose the $\pm$ with the **same sign** as $f'$.

Note also that the quadratic formula can go "crazy" (give complex steps for real functions) if $(f')^2 < 2ff''$.   However, this will never happen if we have a good enough initial guess, so that $f(x)$ is sufficiently small.   In general, higher-order Newton methods tend to be more sensitive to the initial guess!

In [4]:
using ForwardDiff

In [20]:
function newton(f, x, n)
    for i = 1:n
        fx = f(x)
        fx′ = ForwardDiff.derivative(f, x)
        x = x - fx / fx′
    end
    return x
end

newton (generic function with 1 method)

In [32]:
function supernewton(f, x, n)
    for i = 1:n
        fx = f(x)
        fx′ = ForwardDiff.derivative(f, x)
        fx″ = ForwardDiff.derivative(x -> ForwardDiff.derivative(f, x), x)
        x = x - 2fx / (fx′ + copysign(sqrt(fx′^2 - 2fx*fx″), fx′))
    end
    return x
end

supernewton (generic function with 1 method)

In [42]:
f(x) = cos(x) - x
setprecision(BigFloat, 40000)
for i = 1:10
    println(Float64(log10(abs(f(supernewton(f, big(1), i))))))
end

-2.6305807789924294
-9.512351136373354
-30.157759191560796
-92.09398335713593
-277.90265585386135
-835.3286733440375
-2507.6067258145663
-7524.4408832261515
-12041.199826559248
-12041.199826559248


We roughly **triple** the number of digits (**cube** the error) on each step.

Basically, this is because expanding to second order neglects the $O(\delta^3)$ term in the Taylor expansion, so it solves $f(x+\delta)=0$ up to an $O(\delta^3)$ error.